In [25]:
from z3 import *

# Создание тактики для оптимизации портфеля
portfolio_solver = Then(
    With('simplify', mul2concat=True),
    'normalize-bounds',
    'lia2pb',
    'pb2bv',
    'bit-blast',
    'sat'
).solver()

# Определение переменных
stocks, bonds, real_estate = Reals('stocks bonds real_estate')
risk, return_rate = Reals('risk return_rate')

# Добавление ограничений
portfolio_solver.add(stocks >= 0, bonds >= 0, real_estate >= 0)
portfolio_solver.add(stocks + bonds + real_estate == 1)  # Общая аллокация должна быть 1
portfolio_solver.add(risk <= 0.05)  # Лимит на риск
portfolio_solver.add(return_rate >= 0.1)  # Минимальная норма прибыли

# Уравнения, связывающие инвестиции с риском и доходностью
portfolio_solver.add(risk == 0.1*stocks + 0.05*bonds + 0.2*real_estate)
portfolio_solver.add(return_rate == 0.15*stocks + 0.07*bonds + 0.1*real_estate)

# Проверка решения
if portfolio_solver.check() == sat:
    model = portfolio_solver.model()
    print("Solution found:")
    for d in model.decls():
        print(f"{d.name()} = {model[d]}")
else:
    print("No solution found.")

# Альтернативная задача портфеля
alternative_solver = Then(
    With('simplify', mul2concat=True),
    'solve-eqs',
    'bit-blast',
    'aig',  # Использование инвертированных графов для сжатия логических формул
    'sat'
).solver()

# Определение переменных
tech_stocks, healthcare_stocks, bonds = Reals('tech_stocks healthcare_stocks bonds')
risk, return_rate = Reals('risk return_rate')

# Добавление ограничений
alternative_solver.add(tech_stocks >= 0, healthcare_stocks >= 0, bonds >= 0)
alternative_solver.add(tech_stocks + healthcare_stocks + bonds == 1)  # Общая аллокация должна быть 1
alternative_solver.add(risk <= 0.07)  # Лимит на риск
alternative_solver.add(return_rate >= 0.12)  # Минимальная норма прибыли

# Уравнения, связывающие инвестиции с риском и доходностью
alternative_solver.add(risk == 0.12*tech_stocks + 0.06*healthcare_stocks + 0.03*bonds)
alternative_solver.add(return_rate == 0.18*tech_stocks + 0.1*healthcare_stocks + 0.05*bonds)

# Проверка решения
if alternative_solver.check() == sat:
    model = alternative_solver.model()
    print("Alternative solution found:")
    for d in model.decls():
        print(f"{d.name()} = {model[d]}")
else:
    print("No alternative solution found.")

Original Formula: Or(And(a, Not(b)), Or(b, And(Not(a), c)))
Simplified Formula: Or(c, b, a)
